In [1]:
import globs
import grid
import data_processing
import gmaps
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
from ipywidgets.embed import embed_minimal_html


In [49]:
#data need preparation first
def make_maps(load_file, border, key, verbose=False,path=None):
    squares_basic_data, squares_with_data = data_processing.load_data_from_file(load_file, verbose=verbose)
    gmaps.configure(api_key=key)
    lat_c, lon_c = grid.calculate_center_point(border)
    
    #TODO: make function from this
    gradient_google_maps = [(0,0,0,0.2),(0, 255,0,0.8), (255, 255,0,0.8),(255, 0,0,0.8)]
    tmp = [(rgba[0]/255,rgba[1]/255,rgba[2]/255,rgba[3]) for rgba in gradient_google_maps]
    gradient_matplotlib_bar = []
    ticks = 200
    for i in range(len(tmp) - 1):
        if i == 0:
            gradient_matplotlib_bar.append(tmp[0])
        current = tmp[i]
        following = tmp[i+1]
        gradient = []
        for j in range(4):
            gradient.append((following[j]-current[j])/ticks)
        for t in range(1,ticks):
            rgba_tmp = []
            for j in range(4):
                rgba_tmp.append(current[j] + t*gradient[j])
            gradient_matplotlib_bar.append(tuple(rgba_tmp))
    
    locations = []
    stds = []
    means = []
    mins = []
    maxs = []
    for pol, mean, std, min2, max2 in squares_basic_data:
        locations.append(grid.calculate_center_point(pol))
        stds.append(std)
        means.append(mean)
        mins.append(min2)
        maxs.append(max2)

    figs = []
    for fig_type, data in [("Min", mins), ("Max", maxs), ("Mean", means), ("Std", stds)]:
        fig = gmaps.figure(center=(lat_c, lon_c), zoom_level=9)
        #TODO: consider adding max_intensity, different for min/max/mean/std
        heat_layer = gmaps.heatmap_layer(locations=locations, weights=data, point_radius=35)
        heat_layer.gradient = gradient_google_maps
        fig.add_layer(heat_layer)
        
        bar, ax = plt.subplots(figsize=(1, 6))
        bar.subplots_adjust(wspace=0.3, hspace=0.0,right = 0.5)

        cmap = mpl.colors.ListedColormap(colors=gradient_matplotlib_bar)
        norm = mpl.colors.Normalize(vmin=min(data), vmax=max(data))

        bar.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=ax, orientation='vertical', label='Unit')
        bar.add_axes(ax)
        figs.append((fig,bar))
        
        # TODO: add saving chart
        if path is not None:
            # path_to_maps/City_name <- passed as path argument
            # path_to_maps/City_name/Map_type/bar_type.png
            # path_to_maps/City_name/Map_type/map_type.png
            main_dir = path+fig_type + '/'
            if not os.path.exists(path):
                os.mkdir(path)
            if not os.path.exists(main_dir):
                os.mkdir(main_dir)
            # now i ned to save map and bar
            embed_minimal_html(main_dir+fig_type+'_map.html', views=[fig])
            plt.savefig(main_dir+fig_type+'_bar.png')           
        plt.clf()
    return figs


In [50]:
file = globs.cities.TOKYO_FILE
border = globs.cities.TOKYO
p = globs.cities.TOKYO_MAP_DIR

key = ""

figs = make_maps(file, border, key, path=p)


<Figure size 72x432 with 0 Axes>

<Figure size 72x432 with 0 Axes>

<Figure size 72x432 with 0 Axes>

<Figure size 72x432 with 0 Axes>